# HOLA AMIGOS

In [ ]:
!rm -rf datalab
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz /Here_the_directory_you_want_to_put_in
!uncompress aclImdb_v1.tar.gz
!tar -xvf aclImdb_v1.tar
!rm aclImdb_v1.tar

In [3]:
% reset
import os
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
def write_clean_docs(src_dir,target_dir):
    current_doc = None
    directory = os.fsencode(src_dir)
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        with open(os.path.join(src_dir,filename),'r') as f:
            current_doc = f.read()
        with open(os.path.join(target_dir,filename),'w') as f:
            f.write(clean_doc(current_doc))
        

def clean_doc(doc):
    lemmatizer = WordNetLemmatizer()
    clean_doc = re.sub(r'[!"#$%&\'()=*+,-./:;?@\[\]^_`{|}~]|[0-9]|<.+>', ' ', doc)
    clean_doc = ' '.join([word.lower() for word in clean_doc.split()])
    tokens =  [word for word in clean_doc.split() if word not in set(stopwords.words('english'))]
    tagged_tokens = [(pair[0],get_wordnet_pos(pair[1])) for pair in nltk.pos_tag(tokens)]
    clean_doc = ' '.join([ lemmatizer.lemmatize(word,tag) for word,tag in tagged_tokens])
    return clean_doc
    

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [ ]:
# write_clean_docs('Data/aclImdb/train/pos','Data/cleaned_data/train/pos')
# write_clean_docs('Data/aclImdb/train/neg','Data/cleaned_data/train/neg')
# write_clean_docs('Data/aclImdb/train/unsup','Data/cleaned_data/train/unsup')
write_clean_docs('Data/subset','Data/clean_subset')

In [5]:
%reset
from pathlib import Path
import os
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        pathlist = Path(self.dirname).glob('**/*')
        for path in pathlist:
            path_in_str = str(path)
            if os.path.isfile(path_in_str):
                for line in open(path_in_str):
                    yield [w.lower() for w in line.split()]

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [6]:
class MyDocs(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        pathlist = Path(self.dirname).glob('**/*')
        for path in pathlist:
            path_in_str = str(path)
            if os.path.isfile(path_in_str):
                f=open(path_in_str)
                yield [w.lower() for w in f.read().split()]

In [7]:
from gensim.models import Word2Vec
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

sentence = MySentences('Data/cleaned_data/train/pos')

wv_model = Word2Vec(sentence, size=100, window=5, min_count=100, workers=6)

In [8]:
docs = MyDocs('Data/cleaned_data/train/pos')
dct = Dictionary(docs)
corpus = [dct.doc2bow(line) for line in docs]
tf_model = TfidfModel(corpus)
vector = tf_model[corpus[0]]

In [9]:
raw_corpus = [doc for doc in docs]

In [10]:
wv_model.similar_by_word("awesome")

/home/mohamedmahmoud/myprograms/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.


[('fantastic', 0.8176398277282715),
 ('hilarious', 0.7408750057220459),
 ('amazing', 0.7405626773834229),
 ('ok', 0.7380691170692444),
 ('incredible', 0.7281054854393005),
 ('cool', 0.7279859781265259),
 ('absolutely', 0.7059080004692078),
 ('okay', 0.6849677562713623),
 ('plus', 0.6837688088417053),
 ('fabulous', 0.6790148019790649)]